### Search Engine With Tools And Agents

In [6]:
## Arxiv--Research
## Tools crrapper,ArxivAPIWrappereation

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [7]:
# wiki_client > The tool that fetches the raw data from Wikipedia.
#api_wrapper_wiki > A layer that sets rules (like max characters) and cleans the wiki_client's raw data for you

In [8]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250,wiki_client=None)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [9]:
from langchain_community.tools import DuckDuckGoSearchResults

# Create the tool using DuckDuckGoSearchResults
duckduckgo_search = DuckDuckGoSearchResults()

# Use the DuckDuckGoSearchResults instance directly as a tool (it already provides name/invoke)
duck_tool = duckduckgo_search

print(duck_tool.name)


duckduckgo_results_json


In [12]:
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import Tool

# Create the tool using DuckDuckGoSearchResults
duckduckgo_search = DuckDuckGoSearchResults()

# Wrap it into a LangChain Tool
duck_tool = Tool(
    name="DuckDuckGo_Search",
    func=duckduckgo_search.invoke,   # This is how you call the tool
    description="Useful for searching the web using DuckDuckGo."
)

print(duck_tool.name)


ImportError: cannot import name 'Tool' from 'langchain.tools' (c:\Users\Shivam\anaconda3\Lib\site-packages\langchain\tools\__init__.py)

In [ ]:
api_wrapper_arxiv=ArxivAPIWrapper(arxiv_search=None,
    arxiv_exceptions=None,top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [ ]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
#loader=WebBaseLoader("https://docs.smith.langchain.com/")
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings())
retriever=vectordb.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002579E2C02F0>, search_kwargs={})

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"Langsmith_search_tool","Search any information about Langsmith")

retriever_tool.name

'Langsmith_search_tool'

In [ ]:
# let's convert rag into tool

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from pathlib import Path

# Step 1: Resolve and validate PDF path (robust)
pdf_path = Path(r"C:\Agentic_AI_Workshop\2-LangchainConti\2.1-DataIngestion\syllabus.pdf")

if not pdf_path.exists():
	# First try to find a syllabus file
	candidates = list(Path.cwd().rglob("syllabus*.pdf"))
	if not candidates:
		# Fallback: try any PDF in the workspace
		candidates = list(Path.cwd().rglob("*.pdf"))
	if candidates:
		pdf_path = candidates[0]
		print(f"Using found file: {pdf_path}")
	else:
		# Ask the user for an alternate path or offer to download a sample PDF
		user_input = input(
			f"PDF not found at {pdf_path}.\n"
			"Enter full path to a PDF file, or press Enter to download a small sample PDF into the current directory: "
		).strip()
		if user_input:
			alt_path = Path(user_input)
			if alt_path.exists():
				pdf_path = alt_path
				print(f"Using provided file: {pdf_path}")
			else:
				raise FileNotFoundError(f"Provided path does not exist: {alt_path}")
		else:
			# Download a small sample PDF
			sample_url = "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf"
			sample_path = Path.cwd() / "sample_syllabus.pdf"
			try:
				import urllib.request
				print(f"Downloading sample PDF from {sample_url} to {sample_path} ...")
				urllib.request.urlretrieve(sample_url, sample_path)
				pdf_path = sample_path
				print(f"Downloaded sample PDF to {pdf_path}")
			except Exception as e:
				raise FileNotFoundError(f"No PDF found and failed to download sample: {e}")

loader = PyPDFLoader(str(pdf_path))
docs = loader.load()

# Step 2: Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

# Step 3: Create embeddings
embedding_model = HuggingFaceEmbeddings()

# Step 4: Build vector database
vectordb1 = FAISS.from_documents(documents, embedding_model)

# Step 5: Create retriever
retriever1 = vectordb1.as_retriever()


Using found file: c:\Users\Shivam\OneDrive\文档\GitHub\Multi-Tool-Agentic-AI-Assistant-main\data\Agent Quality Whitepaper.pdf


C:\Users\Shivam\AppData\Local\Temp\ipykernel_10900\3289048786.py:53: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings()
C:\Users\Shivam\AppData\Local\Temp\ipykernel_10900\3289048786.py:53: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()


In [ ]:
# from langchain_community.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import HuggingFaceEmbeddings

# # Step 1: Load PDF

# loader = PyPDFLoader(r"C:\Agentic_AI_Workshop\2-LangchainConti\2.1-DataIngestion\syllabus.pdf")
# docs = loader.load()

# # Step 2: Split into chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# documents = text_splitter.split_documents(docs)

# # Step 3: Create embeddings
# embedding_model = HuggingFaceEmbeddings()

# # Step 4: Build vector database
# vectordb1 = FAISS.from_documents(documents, embedding_model)

# # Step 5: Create retriever
# retriever1 = vectordb1.as_retriever()


In [ ]:
from langchain.tools.retriever import create_retriever_tool
pdf_retriever_tool=create_retriever_tool(retriever1,"pdf_retriever_tool","give information about the syllabus pdf")
pdf_retriever_tool.name

'pdf_retriever_tool'

In [ ]:
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

from langchain_community.document_loaders import TextLoader

# Robustly resolve the about_me file path (fall back to workspace search, prompt, or create a sample)
text_path = Path(r"C:\Agentic_AI_Workshop\1-Langchain\about_me.txt")

if not text_path.exists():
	# look for obvious candidates first
	candidates = list(Path.cwd().rglob("about_me*.txt"))
	if candidates:
		text_path = candidates[0]
		print(f"Using found file: {text_path}")
	else:
		# fallback to any txt in repo
		candidates = list(Path.cwd().rglob("*.txt"))
		if candidates:
			text_path = candidates[0]
			print(f"No 'about_me.txt' found. Using first .txt found: {text_path}")
		else:
			# ask user for alternate path or create a sample file
			user_input = input(
				f"Text file not found at {text_path}.\n"
				"Enter full path to a text file, or press Enter to create a small sample file in the current directory: "
			).strip()
			if user_input:
				alt = Path(user_input)
				if alt.exists():
					text_path = alt
					print(f"Using provided file: {text_path}")
				else:
					raise FileNotFoundError(f"Provided path does not exist: {alt}")
			else:
				sample_path = Path.cwd() / "sample_about_me.txt"
				sample_path.write_text(
					"Name: Abhiram\nBio: Software engineer interested in LLMs and agents.\n", encoding="utf8"
				)
				text_path = sample_path
				print(f"Created sample file at {sample_path}")

# Load the file with explicit encoding and provide clear error if it fails
loader1 = TextLoader(str(text_path), encoding="utf8")
try:
	docs1 = loader1.load()
except Exception as e:
	raise RuntimeError(f"Error loading {text_path}: {e}")

# Step 2: Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs1)

# Step 3: Create embeddings
embedding_mode2 = HuggingFaceEmbeddings()

# Step 4: Build vector database
vectordb2 = FAISS.from_documents(documents, embedding_mode2)

# Step 5: Create retriever
retriever2 = vectordb2.as_retriever()

Using found file: c:\Users\Shivam\OneDrive\文档\GitHub\Multi-Tool-Agentic-AI-Assistant-main\data\about_me.txt


C:\Users\Shivam\AppData\Local\Temp\ipykernel_10900\284276796.py:55: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_mode2 = HuggingFaceEmbeddings()


In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings

# from langchain_community.document_loaders import TextLoader

# # loader=TextLoader("C:\Agentic_Ai\Agentic_AI_Workspace\1-Langchain\about_me.txt")
# # docs = loader.load()
# loader1 = TextLoader(r"C:\Agentic_AI_Workshop\1-Langchain\about_me.txt")
# docs1 = loader1.load()

# # Step 2: Split into chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# documents = text_splitter.split_documents(docs1)

# # Step 3: Create embeddings
# embedding_mode2 = HuggingFaceEmbeddings()

# # Step 4: Build vector database
# vectordb2 = FAISS.from_documents(documents, embedding_mode2)

# # Step 5: Create retriever
# retriever2 = vectordb2.as_retriever()

In [ ]:
# text_retriever_tool=create_retriever_tool(retriever2,"Text_retriever_tool","give information about the syllabus pdf")
# text_retriever_tool.name

In [ ]:
# from langchain.chains import RetrievalQA
# from langchain_community.llms import Ollama
# from langchain_community.embeddings import OllamaEmbeddings
# from langchain_community.vectorstores import FAISS

# # Initialize Ollama LLM (use an available model like mistral, llama3, etc.)

# # Use FAISS as a retriever
# retriever = vectordb2.as_retriever()

# # Create a RetrievalQA chain
# qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

# # Ask a query
# query = input("Enter your query: ")
# answer = qa_chain.run(query)

# print(answer)


In [ ]:
from langchain.tools.retriever import create_retriever_tool
text_retriever_tool=create_retriever_tool(retriever2,"text_reteriever_tool","give information about the person mentioned in the text file")
text_retriever_tool.name

'text_reteriever_tool'

In [ ]:
tools=[text_retriever_tool,wiki,arxiv,duck_tool,pdf_retriever_tool]


In [ ]:
# tools=[wiki,arxiv,duck_tool,retriever_tool,pdf_retriever_tool]

In [ ]:
tools

[Tool(name='text_reteriever_tool', description='give information about the person mentioned in the text file', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000257B3824E00>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000257B52AD950>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000257B38262A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000257B52AD950>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, tem

In [ ]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

# groq_api_key=os.getenv("GROQ_API_KEY")
# #openai.api_key=os.getenv("OPENAI_API_KEY")

# llm=ChatGroq(api_key=groq_api_key,
#              model_name="gemma2-9b-it"
#              )

In [ ]:
api_key=os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq

# Ensure we don't set an environment variable to None (which raises TypeError).
# Use existing api_key if present, otherwise try env var, otherwise prompt the user.
if not api_key:
    api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    api_key = input("Enter your GROQ API key (or set GROQ_API_KEY env var): ").strip()
    if not api_key:
        raise RuntimeError("GROQ_API_KEY is not set. Please set the environment variable or provide a key.")

# Now it's safe to set the environment variable
os.environ["GROQ_API_KEY"] = api_key

llm = ChatGroq(
    api_key=api_key,
    model="llama-3.1-8b-instant"
)


In [ ]:
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY") # type: ignore
llm = ChatGroq(
    api_key=api_key,
    model="llama-3.1-8b-instant"
)


In [ ]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\Shivam\OneDrive\文档\GitHub\Multi-Tool-Agentic-AI-Assistant-main\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant."),
#     MessagesPlaceholder("chat_history", optional=True),
#     ("human", "{input}"),
#     MessagesPlaceholder("agent_scratchpad"),
# ])

In [ ]:
# from langchain.agents import create_agent
# agent=create_agent(llm,tools,prompt=prompt)


In [ ]:
# Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [ ]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [ ]:
# Verify tools are correct
print(f"Number of tools: {len(tools)}")
print(f"Tool names: {[tool.name for tool in tools]}")

# Should output:
# Number of tools: 5
# Tool names: ['wikipedia', 'arxiv', 'DuckDuckGo_Search', 'pdf_retriever_tool', 'text_retriever_tool']

Number of tools: 5
Tool names: ['text_reteriever_tool', 'wikipedia', 'arxiv', 'DuckDuckGo_Search', 'pdf_retriever_tool']


In [ ]:
agent_executor.invoke({"input":"Where is Shivam from?"})



> Entering new AgentExecutor chain...


APIError: tool call validation failed: attempted to call tool 'wiki_arxiv_tool' which was not in request.tools

In [ ]:
agent_executor.invoke({"input":"what is the langsmith?"})



> Entering new AgentExecutor chain...


APIError: Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.

In [ ]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`




c:\Users\Shivam\OneDrive\文档\GitHub\Multi-Tool-Agentic-AI-Assistant-main\.venv\Lib\site-packages\langchain_community\utilities\arxiv.py:102: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cI'll stop the response here since the information is already out of date

> Finished chain.


{'input': "What's the paper 1706.03762 about?",
 'output': "I'll stop the response here since the information is already out of date"}

In [ ]:
agent_executor.invoke({"input":"what is Gererative AI?"
""})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Generative AI'}`




c:\Users\Shivam\OneDrive\文档\GitHub\Multi-Tool-Agentic-AI-Assistant-main\.venv\Lib\site-packages\langchain_community\utilities\arxiv.py:105: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2025-01-06
Title: Foundations of GenIR
Authors: Qingyao Ai, Jingtao Zhan, Yiqun Liu
Summary: The chapter discusses the foundational impact of modern generative AI models on information access (IA) systems. In contrast to traditional AI, th
Invoking: `wikipedia` with `{'query': 'Generative AI'}`


Page: Generative artificial intelligence
Summary: Generative artificial intelligence (Generative AI or GenAI) is a subfield of artificial intelligence that uses generative models to generate text, images, videos, audio, software code or other forms o
Invoking: `wikipedia` with `{'query': 'Generative AI applications'}`


Page: Generative AI pornography
Summary: Generative AI pornography or simply AI pornography is a digitally created pornography produced through generative artificial intelligence (AI) technologies. Unlike traditional pornography, which involves real 
Invoking: `wikipedia` with `{'query': 'Generative AI laws'}`


Page: Generative AI pornography
Summary: Generative AI 

{'input': 'what is Gererative AI?',
 'output': 'The information about Generative AI can be found in the previously provided links.'}